In [1]:
# Install the required libraries for the MediWaste AI Engine
!pip install tensorflow joblib numpy Pillow scikit-learn

   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/332.0 MB 5.9 MB/s eta 0:00:57
   ---------------------------------------- 2.4/332.0 MB 5.4 MB/s eta 0:01:01
   ---------------------------------------- 3.4/332.0 MB 5.3 MB/s eta 0:01:02
    --------------------------------------- 4.5/332.0 MB 5.3 MB/s eta 0:01:02
    --------------------------------------- 5.5/332.0 MB 5.3 MB/s eta 0:01:02
    --------------------------------------- 6.6/332.0 MB 5.2 MB/s eta 0:01:03
    --------------------------------------- 7.6/332.0 MB 5.2 MB/s eta 0:01:03
   - -------------------------------------- 8.7/332.0 MB 5.2 MB/s eta 0:01:03
   - -------------------------------------- 8.7/332.0 MB 5.2 MB/s eta 0:01:03
   - -------------------------------------- 10.7/332.0 MB 5.1 MB/s eta 0:01:03
   - -------------------------------------- 11.8/332.0 MB 5.2 MB/s eta 0:01:0

In [13]:
# MediWaste AI: Portable Inference Script (Robust Version)
# This script assumes ALL model files are in the SAME folder as this file.

import os
import sys
import warnings
import tensorflow as tf
import joblib
import numpy as np
from PIL import Image

# --- 0. SUPPRESS VERSION WARNINGS ---
# Suppresses scikit-learn unpickling warnings caused by minor version mismatches
# (e.g., training on 1.6.1 and running on 1.7.1)
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# --- 1. ROBUST PATH HANDLING ---
try:
    # Works if running as a .py script
    CURRENT_FOLDER = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Fallback for Jupyter Notebooks
    CURRENT_FOLDER = os.getcwd()

# Define paths for the three required artifacts
MODEL_PATH = os.path.join(CURRENT_FOLDER, 'efficientnet_finetuned_model.keras')
SCALER_PATH = os.path.join(CURRENT_FOLDER, 'rf_finetuned_features_scaler.joblib')
BRAIN_PATH = os.path.join(CURRENT_FOLDER, 'rf_finetuned_features_classifier.joblib')

# Global variables initialized as None
feature_extractor = None
scaler = None
classifier = None

def verify_and_load():
    global feature_extractor, scaler, classifier
    
    print(f"Checking directory: {CURRENT_FOLDER}")
    missing = []
    for f in [MODEL_PATH, SCALER_PATH, BRAIN_PATH]:
        if not os.path.exists(f):
            missing.append(os.path.basename(f))
    
    if missing:
        print("\n" + "!"*50)
        print("CRITICAL ERROR: AI MODEL FILES NOT FOUND")
        print(f"Please move the following files into this folder:")
        for m in missing: print(f"  -> {m}")
        print("!"*50 + "\n")
        return False

    try:
        print("Waking up Vision Engine...")
        # Load the full Keras model
        vision_model = tf.keras.models.load_model(MODEL_PATH)
        
        # Extract the feature layer (avg_pool) to use as our visual descriptor
        feature_extractor = tf.keras.Model(
            inputs=vision_model.input, 
            outputs=vision_model.get_layer('avg_pool').output
        )

        print("Loading Hybrid Brain...")
        scaler = joblib.load(SCALER_PATH)
        classifier = joblib.load(BRAIN_PATH)
        
        print("[✓] All systems online and ready for inference.\n")
        return True
    except Exception as e:
        print(f"Error during model loading: {e}")
        return False

# Category Mapping
CLASSES = [
    '(ME) Metal', '(OW) Organic', '(PE) Plastic', '(PP) Paper', 
    '(SN) Needle', 'Body Tissue', 'Gauze', 'Glass', 
    'Gloves', 'Mask', 'Syringe', 'Tweezers'
]

def quick_predict(image_name):
    """
    Performs the full inference pipeline on a local image file.
    """
    if feature_extractor is None:
        print("Cannot predict: Models were not loaded correctly.")
        return

    img_path = os.path.join(CURRENT_FOLDER, image_name)
    if not os.path.exists(img_path):
        print(f"Error: Could not find image '{image_name}' in {CURRENT_FOLDER}")
        return

    try:
        # 1. Pre-processing: Standardize for EfficientNetV2 (224x224)
        img = Image.open(img_path).convert('RGB').resize((224, 224))
        img_array = np.expand_dims(np.array(img), axis=0)

        # 2. Vision Extraction: Get 1,280 neural features
        features = feature_extractor.predict(img_array, verbose=0)

        # 3. Scaling: Normalize features using the pre-trained scaler
        scaled_features = scaler.transform(features)

        # 4. Classification: Identify using the Random Forest Brain
        prediction = classifier.predict(scaled_features)[0]
        
        # 5. Confidence: Extract probability
        probs = classifier.predict_proba(scaled_features)[0]
        confidence = probs[prediction] * 100
        
        print(f"\n--- Result for {image_name} ---")
        print(f"IDENTIFIED : {CLASSES[prediction]}")
        print(f"CONFIDENCE : {confidence:.2f}%")
        print("-" * 30)
    except Exception as e:
        print(f"Error during inference: {e}")

# --- EXECUTION GATE ---
# 1. Attempt to load the AI pipeline
models_ready = verify_and_load()

# 2. If successful, run test prediction
if __name__ == "__main__":
    if models_ready:
        # Default test file name
        test_file = "test.jpg" 
        if os.path.exists(os.path.join(CURRENT_FOLDER, test_file)):
            quick_predict(test_file)
        else:
            print(f"System ready. Place an image named '{test_file}' in the folder to test.")
    else:
        print("Fix the missing files issue above to start analysis.")

Checking directory: C:\Users\aryan\OneDrive\Documents\MediWaste_AI\AI_Scanner
Waking up Vision Engine...
Loading Hybrid Brain...
[✓] All systems online and ready for inference.


--- Result for test.jpg ---
IDENTIFIED : Syringe
CONFIDENCE : 93.00%
------------------------------
